# RDT 实验结果分析与可视化 (Analysis & Visualization)

## 1 设置环境，导入必要的库，并指定实验结果文件夹。



In [17]:
# a = 1 # ipynb 单元格占位符
# --- 基本设置与导入 ---
import os
import sys
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

# --- 添加 src 目录到 Python 路径 (如果 Notebook 不在 rdt_framework 根目录) ---
# 获取当前 Notebook 文件所在的目录
notebook_dir = os.path.dirname(os.path.abspath("__file__")) # 在 Notebook 中，__file__ 可能不可用，换一种方式
# notebook_dir = os.getcwd() # 或者使用当前工作目录

# 假设 Notebook 在 rdt_framework 根目录下，或者 src 与 Notebook 在同一层级
# 如果 src 在上一级目录，可以使用 os.path.join(notebook_dir, '..')
project_root = os.path.join(notebook_dir, '..') if 'src' not in os.listdir(notebook_dir) else notebook_dir
# src_path = os.path.join(project_root, 'src')
src_path = os.path.join(os.getcwd(), 'src') # 假设 notebook 和 src 在同一级
if src_path not in sys.path:
    sys.path.append(os.path.dirname(src_path)) # 将 src 的父目录加入，以便 import src.xxx

print(f"Project Root (estimated): {project_root}")
print(f"Src Path: {src_path}")
print(f"System Path contains src parent: {os.path.dirname(src_path) in sys.path}")

# --- 导入自定义模块 ---
try:
    from src import config as default_config # 加载默认配置以获取模型结构等信息
    from src import utils
    from src.models import get_model # 使用 get_model 来加载模型
    from src.data_handler import load_and_preprocess_data # 用于加载数据
    print("自定义模块导入成功。")
except ImportError as e:
    print(f"错误：无法导入自定义模块。请确保 Notebook 可以访问 'src' 目录。")
    print(f"ImportError: {e}")
    # 这里可以停止执行或提供更详细的指引
    raise e
except Exception as e:
    print(f"导入自定义模块时发生未知错误: {e}")
    raise e


# --- 配置 Matplotlib 和 Seaborn 样式 ---
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("tab10")
# 设置中文字体 (如果需要显示中文)
# plt.rcParams['font.sans-serif'] = ['SimHei'] # 指定默认字体
# plt.rcParams['axes.unicode_minus'] = False   # 解决保存图像是负号'-'显示为方块的问题


# --- 用户指定：实验结果文件夹路径 ---
# !!! 重要：请将下面的路径替换为您实际的实验结果文件夹路径 !!!
# 例如: 'results/weatherHistory_DLinear_PatchTST_20231027_103000'
EXPERIMENT_RESULTS_DIR = 'results/weatherHistory0501'

# --- 构建必要的子目录路径 ---
METRICS_DIR = os.path.join(EXPERIMENT_RESULTS_DIR, 'metrics')
MODELS_DIR = os.path.join(EXPERIMENT_RESULTS_DIR, 'models')
PLOTS_DIR = os.path.join(EXPERIMENT_RESULTS_DIR, 'plots') # Note: Notebook 中生成的图也可以直接显示

# --- 检查路径是否存在 ---
if not os.path.isdir(EXPERIMENT_RESULTS_DIR):
    print(f"错误：指定的实验结果文件夹不存在: {EXPERIMENT_RESULTS_DIR}")
    # 可以在这里停止执行
    raise FileNotFoundError(f"实验结果文件夹未找到: {EXPERIMENT_RESULTS_DIR}")
else:
    print(f"使用实验结果文件夹: {EXPERIMENT_RESULTS_DIR}")
    print(f"指标目录: {METRICS_DIR}")
    print(f"模型目录: {MODELS_DIR}")

# --- 设备设置 ---
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"使用设备: {DEVICE}")



Project Root (estimated): d:\OneDrive\Project\Reverse_Distill_Training
Src Path: d:\OneDrive\Project\Reverse_Distill_Training\src
System Path contains src parent: False
自定义模块导入成功。
使用实验结果文件夹: results/weatherHistory0501
指标目录: results/weatherHistory0501\metrics
模型目录: results/weatherHistory0501\models
使用设备: cuda


## 2. 读取实验总结性指标文件。

In [ ]:
# a = 1 # ipynb 单元格占位符
# --- 读取实验总结 CSV 文件 ---

# 查找 metrics 目录下的 all_runs_summary_*.csv 文件
summary_files = [f for f in os.listdir(METRICS_DIR) if f.startswith('all_runs_summary') and f.endswith('.csv')]

if not summary_files:
    print(f"错误：在 {METRICS_DIR} 中未找到 'all_runs_summary_*.csv' 文件。")
    # 可以在这里停止执行
    results = []


错误：在 results/weatherHistory0501\metrics 中未找到 'all_runs_summary_*.csv' 文件。


## 3. 定义模型加载函数和参数计算函数。

In [22]:
# a = 1 # ipynb 单元格占位符
# --- 模型加载与参数计算 ---

def load_trained_model(model_name, model_config, model_path, device):
    """加载指定路径的预训练模型"""
    print(f"\n正在加载模型: {model_name} 从 {model_path}")
    # 使用 get_model 初始化模型结构，需要确保 model_config 正确
    # 注意：这里的 model_config 应该与训练时使用的配置严格一致！
    try:
        # 确保 n_series, input_size, h 等关键参数在配置中是最新的
        # 我们从 default_config 更新这些依赖数据的参数
        model_config['n_series'] = len(default_config.TARGET_COLS)
        model_config['input_size'] = default_config.LOOKBACK_WINDOW
        model_config['h'] = default_config.PREDICTION_HORIZON
        
        model = get_model(model_name, model_config.copy()) # 使用配置的副本
    except Exception as e:
        print(f"错误：初始化模型 '{model_name}' 失败。检查配置是否正确。错误：{e}")
        return None

    # 加载模型权重
    loaded_model = utils.load_model(model, model_path, device)
    if loaded_model:
        print(f"模型 '{model_name}' 加载成功。")
    else:
        print(f"警告：加载模型 '{model_name}' 失败，路径可能无效或文件损坏。")
    return loaded_model

def count_parameters(model):
    """计算模型的可训练参数数量"""
    if model is None:
        return 0
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# --- 加载所有模型 ---
models = {}
model_configs = {
    'Teacher': (default_config.TEACHER_MODEL_NAME, default_config.TEACHER_CONFIG),
    'Student_TaskOnly': (default_config.STUDENT_MODEL_NAME, default_config.STUDENT_CONFIG),
    'Student_RDT': (default_config.STUDENT_MODEL_NAME, default_config.STUDENT_CONFIG),
    'Student_Follower': (default_config.STUDENT_MODEL_NAME, default_config.STUDENT_CONFIG)
}
model_paths = {
    'Teacher': run_data.get('teacher_model_path'),
    'Student_TaskOnly': run_data.get('student_task_only_model_path'),
    'Student_RDT': run_data.get('student_rdt_model_path'),
    'Student_Follower': run_data.get('student_follower_model_path')
}

model_params = {}
model_metrics = {}

for name, (model_name_str, model_config_dict) in model_configs.items():
    model_path = model_paths.get(name)
    if model_path and isinstance(model_path, str) and os.path.exists(model_path):
        model = load_trained_model(model_name_str, model_config_dict, model_path, DEVICE)
        models[name] = model
        # 计算参数量
        params = count_parameters(model)
        model_params[name] = params
        print(f"{name} ({model_name_str}) 参数量: {params:,}")
        
        # 提取性能指标 (MSE, MAE)
        metrics = {}
        for metric in default_config.METRICS: # 'mae', 'mse'
             col_name = f"{name}_{metric}"
             if col_name in run_data:
                 metrics[metric.upper()] = run_data[col_name]
             else:
                 print(f"警告：在结果中未找到指标 '{col_name}'")
                 metrics[metric.upper()] = np.nan # 或者 0
        model_metrics[name] = metrics
        print(f"{name} 指标: {metrics}")
    else:
        print(f"\n警告：模型 '{name}' 的路径未找到或无效 ({model_path})，跳过加载。")
        models[name] = None
        model_params[name] = 0
        model_metrics[name] = {m.upper(): np.nan for m in default_config.METRICS}



NameError: name 'run_data' is not defined

## 4.可视化模型参数量与性能指标（MSE/MAE）的关系。

In [ ]:
# a = 1 # ipynb 单元格占位符
# --- 可视化模型参数量与性能 ---

# 准备数据
plot_df_data = []
for name, params in model_params.items():
    if params > 0: # 只包括成功加载并有参数的模型
        metrics = model_metrics.get(name, {})
        plot_df_data.append({
            'Model': name,
            'Parameters': params,
            'MSE': metrics.get('MSE', np.nan),
            'MAE': metrics.get('MAE', np.nan)
        })

if not plot_df_data:
    print("没有有效的模型数据可供绘图。")
else:
    plot_df = pd.DataFrame(plot_df_data)
    print("\n用于绘图的数据:")
    display(plot_df)

    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    fig.suptitle('模型性能 vs 参数量', fontsize=16)

    # 绘制 MSE vs Parameters
    sns.scatterplot(data=plot_df, x='Parameters', y='MSE', hue='Model', s=150, ax=axes[0])
    axes[0].set_title('MSE vs Parameters')
    axes[0].set_xlabel('Number of Parameters')
    axes[0].set_ylabel('Mean Squared Error (MSE)')
    axes[0].ticklabel_format(style='sci', axis='x', scilimits=(0,0)) # 科学计数法显示x轴
    axes[0].grid(True)
    # 在点旁边标注模型名称
    for i in range(plot_df.shape[0]):
     axes[0].text(plot_df['Parameters'][i]*1.01, plot_df['MSE'][i], plot_df['Model'][i], fontsize=9)


    # 绘制 MAE vs Parameters
    sns.scatterplot(data=plot_df, x='Parameters', y='MAE', hue='Model', s=150, ax=axes[1], legend=False) # 不重复显示图例
    axes[1].set_title('MAE vs Parameters')
    axes[1].set_xlabel('Number of Parameters')
    axes[1].set_ylabel('Mean Absolute Error (MAE)')
    axes[1].ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    axes[1].grid(True)
    # 在点旁边标注模型名称
    for i in range(plot_df.shape[0]):
     axes[1].text(plot_df['Parameters'][i]*1.01, plot_df['MAE'][i], plot_df['Model'][i], fontsize=9)


    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # 调整布局防止标题重叠
    plt.show()

    # 也可以用条形图展示参数量
    plt.figure(figsize=(10, 5))
    sns.barplot(data=plot_df, x='Model', y='Parameters', hue='Model', dodge=False)
    plt.title('模型参数量对比')
    plt.ylabel('Number of Parameters')
    plt.xlabel('Model')
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    plt.xticks(rotation=15)
    plt.grid(axis='y')
    plt.show()


## 5. 自定义噪声

In [ ]:
# a = 1 # ipynb 单元格占位符
# --- 可视化输入数据的噪声添加效果 ---

# 1. 加载少量原始数据或测试数据样本
try:
    # 尝试加载测试数据加载器中的一个批次
    # 注意：这会重新执行数据加载和预处理，如果数据量大可能会慢
    _, _, test_loader, _ = load_and_preprocess_data(default_config)
    sample_batch_x, sample_batch_y = next(iter(test_loader)) # 获取一个批次 (标准化后的数据)
    # 选择一个样本进行可视化
    sample_x = sample_batch_x[0].numpy() # [lookback, features]
    print(f"加载了一个测试样本 (标准化后) Shape: {sample_x.shape}")
except Exception as e:
    print(f"加载数据样本失败: {e}. 无法进行噪声可视化。")
    sample_x = None # 设置为 None 以跳过后续绘图

if sample_x is not None:
    # 2. 定义噪声水平
    noise_levels_to_viz = [0.0, 0.05, 0.1, 0.2] # 选择几个噪声水平展示

    # 3. 选择要可视化的特征索引
    feature_idx_to_viz = 0
    if feature_idx_to_viz >= sample_x.shape[1]:
        print(f"警告：选择的特征索引 {feature_idx_to_viz} 超出范围，将使用索引 0。")
        feature_idx_to_viz = 0
    feature_name = default_config.TARGET_COLS[feature_idx_to_viz] if feature_idx_to_viz < len(default_config.TARGET_COLS) else f"Feature {feature_idx_to_viz}"

    # 4. 添加噪声并绘图
    plt.figure(figsize=(15, 8))
    plt.title(f'添加不同级别高斯噪声的效果 (特征: {feature_name}, 标准化后数据)', fontsize=14)
    plt.xlabel('Time Steps (Lookback Window)', fontsize=12)
    plt.ylabel('Scaled Value', fontsize=12)

    # 绘制原始（无噪声）数据
    plt.plot(sample_x[:, feature_idx_to_viz], label='Original (Noise=0.0)', linewidth=2, marker='o', markersize=4)

    # 添加并绘制不同级别的噪声
    for noise_level in noise_levels_to_viz:
        if noise_level > 0:
            # 计算标准差（基于当前样本）
            std_dev = np.std(sample_x[:, feature_idx_to_viz])
            noise_sigma = std_dev * noise_level
            # 生成高斯噪声
            noise = np.random.normal(0, noise_sigma, size=sample_x[:, feature_idx_to_viz].shape)
            noisy_sample = sample_x[:, feature_idx_to_viz] + noise
            plt.plot(noisy_sample, label=f'Noise Level={noise_level:.2f}', alpha=0.7, linestyle='--')

    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()


## 6. 指标汇总

In [ ]:
# a = 1 # ipynb 单元格占位符
# --- 性能指标汇总表 ---

# 准备数据
metrics_summary_data = []
for name, metrics in model_metrics.items():
    if model_params[name] > 0: # 仅包括成功加载的模型
        row = {'Model': name}
        row.update(metrics)
        metrics_summary_data.append(row)

if not metrics_summary_data:
    print("没有有效的模型指标数据可供汇总。")
else:
    metrics_summary_df = pd.DataFrame(metrics_summary_data)
    metrics_summary_df = metrics_summary_df.set_index('Model') # 将模型名设为索引

    # (可选) 添加 MAPE 和 WAPE (如果已在 evaluator.py 中计算并保存在结果里)
    # 假设结果文件中有 'ModelName_mape' 和 'ModelName_wape' 列
    extra_metrics = ['MAPE', 'WAPE'] # 需要计算的额外指标
    for extra_metric in extra_metrics:
        metric_col_data = {}
        metric_present = False
        for model_name in metrics_summary_df.index:
            col_name = f"{model_name}_{extra_metric.lower()}"
            if col_name in run_data:
                metric_col_data[model_name] = run_data[col_name]
                metric_present = True
            else:
                metric_col_data[model_name] = np.nan # 如果不存在则填充 NaN
        
        if metric_present:
            metrics_summary_df[extra_metric] = metrics_summary_df.index.map(metric_col_data)
            print(f"已添加指标: {extra_metric}")
        else:
            print(f"未在结果文件中找到指标 '{extra_metric}' 的数据。")


    print("\n模型性能指标汇总表 (来自选择的运行轮次):")
    # 格式化输出，保留几位小数
    display(metrics_summary_df.style.format("{:.4f}", na_rep="-"))

    # 也可以直接使用之前绘制指标对比图的函数
    # utils.plot_metric_comparison(model_metrics,
    #                              title=f"模型性能指标对比 (Run ID: {target_run_id})",
    #                              save_path=os.path.join(EXPERIMENT_RESULTS_DIR, "analysis_metric_comparison.png"))
    # print(f"指标对比图已保存到: {os.path.join(EXPERIMENT_RESULTS_DIR, 'analysis_metric_comparison.png')}")
    # 或者直接显示
    
    # 重新整理 model_metrics 字典以符合 plot_metric_comparison 的输入格式
    plot_metrics_dict = {}
    for model, metrics in metrics_summary_df.T.to_dict().items(): # 转置再转字典
         # 过滤掉 NaN 值，并将 key 转为小写
        plot_metrics_dict[model] = {k.lower(): v for k, v in metrics.items() if pd.notna(v)}

    if plot_metrics_dict:
        fig, ax = plt.subplots(figsize=(12, 7)) # 创建一个图表对象以便显示
        # 调用绘图函数 (假设 utils.plot_metric_comparison 内部会创建图表)
        # 注意：utils 内的函数默认是保存文件，这里我们可能希望直接显示
        # 可以修改 utils 函数使其返回 fig 对象，或在这里重绘逻辑
        
        # --- 重绘逻辑 (简化版条形图) ---
        df_plot = pd.DataFrame(plot_metrics_dict).T # 转置，模型为行，指标为列
        if not df_plot.empty:
            df_plot.plot(kind='bar', figsize=(12, 7), ax=ax)
            ax.set_title(f"模型性能指标对比 (Run ID: {target_run_id})")
            ax.set_ylabel("Metric Value (Lower is Better)")
            ax.set_xlabel("Model")
            plt.xticks(rotation=15)
            plt.grid(axis='y', linestyle='--', alpha=0.7)
            plt.tight_layout()
            plt.show()
        else:
            print("没有可用于绘制条形图的指标数据。")
    else:
        print("没有有效的指标数据用于绘图。")



## 8. 降维可视化

In [ ]:
# a = 1 # ipynb 单元格占位符
# --- 使用核密度估计 (KDE) 可视化数据分布 ---

# 复用之前为 t-SNE 准备的数据 data_for_tsne (标准化后的)
if data_for_tsne is not None and data_for_tsne.shape[0] > 0:
    kde_data_df = pd.DataFrame(data_for_tsne, columns=default_config.TARGET_COLS)

    num_features = kde_data_df.shape[1]
    if num_features == 0:
        print("错误：无法进行 KDE 可视化，没有找到特征数据。")
    elif num_features == 1:
        # 单变量 KDE
        feature_name = kde_data_df.columns[0]
        plt.figure(figsize=(10, 6))
        sns.kdeplot(data=kde_data_df, x=feature_name, fill=True)
        plt.title(f'核密度估计 (KDE) - 特征: {feature_name} (标准化后)', fontsize=14)
        plt.xlabel('Scaled Value', fontsize=12)
        plt.ylabel('Density', fontsize=12)
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.show()
    else:
        # 多变量 KDE (绘制配对图或选择两个特征绘制二维 KDE)
        print("数据包含多个特征，绘制 KDE 配对图。")
        # 配对图会显示每个特征的单变量 KDE 和特征之间的二维 KDE
        g = sns.pairplot(kde_data_df.iloc[:, :min(num_features, 5)], diag_kind='kde', plot_kws={'alpha': 0.6}) # 最多显示前5个特征
        g.fig.suptitle('KDE 配对图 (部分特征, 标准化后)', y=1.02, fontsize=16) # 调整标题位置
        plt.show()

        # 或者选择前两个特征绘制二维 KDE
        if num_features >= 2:
            feature1 = kde_data_df.columns[0]
            feature2 = kde_data_df.columns[1]
            plt.figure(figsize=(10, 8))
            sns.kdeplot(data=kde_data_df, x=feature1, y=feature2, cmap="Blues", fill=True, thresh=0.05)
            plt.title(f'二维 KDE - {feature1} vs {feature2} (标准化后)', fontsize=14)
            plt.xlabel(f'Scaled {feature1}', fontsize=12)
            plt.ylabel(f'Scaled {feature2}', fontsize=12)
            plt.grid(True, linestyle='--', alpha=0.6)
            plt.show()
else:
    print("没有有效的数据可用于 KDE 可视化。")

